En este Notebook, se presenta un conjunto de pasos para realizar el proceso de Extracción, Carga y Transformación sobre el cojunto de datos users_items con el proposito de construir un MVP (Minimum Viable Product) o Producto Mínimo Viable

Ingesta de Datos (Extracción) Dataset Users items

In [1]:
import pandas as pd
import numpy as np
import ast
import gzip
import re

In [2]:
# Carga del Dataset users_items.json
ruta_archivo = '..\\Datasets\\users_items.json.gz'

json_datos = list()

# Abrir y leer el archivo users_items.json.gz
with gzip.open(ruta_archivo, 'rt', encoding='utf-8') as archivo_json:
    for linea in archivo_json:
        try:
            # Evaluar una cadena que representa un diccionario
            json_data = ast.literal_eval(linea)
            json_datos.append(json_data)
        except ValueError as e:
            print(f"ERROR al leer o extraer : {linea}")
            continue

In [3]:
# Transformar la lista json_datos a un dataframe.
df_items = pd.DataFrame(json_datos)

# Observemos dimension y numero de variables
print(f'Filas : {df_items.shape[0]}, Variables : {df_items.shape[1]}')
df_items.head(2)

Filas : 88310, Variables : 5


,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."


In [4]:
# Observemos cantidad de registros, variables, valores no nulos y tipo de datos 
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


In [5]:
# Observamos dos variables que no nos aporta información para nuestras consignas y la cual seran excluidas del dataframe
variables_excluidas = ['steam_id', 'user_url']
df_items.drop(columns= variables_excluidas, inplace= True)

In [6]:
# Observemos la variable items, debemos tener una fila por cada valor del diccionario
df_items.head(3)

,user_id,items_count,items
0,76561197970982479,277,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,"[{'item_id': '1200', 'item_name': 'Red Orchest..."


In [7]:
# Nos toca "Explotar" la variable items, de tal manera tener cada elemento de la lista en una fila independiente
df_items = df_items.explode('items').reset_index(drop=True)

# Normalizamos los datos del json a una tabla
items = pd.json_normalize(df_items['items']).reset_index(drop=True)

# Realizamos dos pasos, concatenamos nuestro df_items (sin la variable items) con la nueva columna items 
df_items = pd.concat([df_items.drop('items', axis=1), items], axis=1)

In [8]:
# Observar que fueron anexadas nuevas variables perteneciente a estructura de diccionario en items
print(f'Filas : {df_items.shape[0]}, Variables : {df_items.shape[1]}')
df_items.info()

Filas : 5170015, Variables : 6
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 6 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   item_id           object 
 3   item_name         object 
 4   playtime_forever  float64
 5   playtime_2weeks   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 236.7+ MB


In [9]:
# Eliminar dos variables que no nos aporta informacion
variables_excluidas = ['item_id','item_name']
df_items.drop(columns = variables_excluidas, inplace=True)
df_items.head(2)

,user_id,items_count,playtime_forever,playtime_2weeks
0,76561197970982479,277,6.0,0.0
1,76561197970982479,277,0.0,0.0


In [10]:
# Anexar las nuevas variables como parte del dataframe nos hizo que aumentara en fila a 5170015, busquemos duplicados 
# que nos permita reducir el numero de lineas, y luego eliminarlas.
print(f'Cantidad de duplicados : {df_items.duplicated().sum()}')
df_items.drop_duplicates(keep='first',inplace=True)
print(f'Filas : {df_items.shape[0]}, Variables : {df_items.shape[1]}')

Cantidad de duplicados : 2253629
Filas : 2916386, Variables : 4


In [11]:
# Crear un archivo con formato parquet
ruta_archivo = '..\\Datasets\\users_items.parquet'
df_items.to_parquet(ruta_archivo, compression='snappy', index=False)